In [3]:
from dit import DiT, sample
import torch as t
import matplotlib.pyplot as plt

In [4]:
model = DiT(28, 28, patch_size=2, n_classes=11, d=12*16, n_head=12, n_blocks=6)
model.load_state_dict(t.load("/share/u/wendler/code/mnist/checkpoints/model.pt"))

<All keys matched successfully>

In [6]:
model.eval()
model.cuda()

DiT(
  (patch): Patch(
    (conv): Conv2d(1, 192, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  )
  (te): NumEmbedding()
  (ce): Embedding(11, 192)
  (act): SiLU()
  (blocks): ModuleList(
    (0-5): 6 x DiTBlock(
      (norm1): RMSNorm()
      (attn): Attention(
        (QKV): Linear(in_features=192, out_features=576, bias=False)
        (O): Linear(in_features=192, out_features=192, bias=False)
        (normq): RMSNorm()
        (normk): RMSNorm()
      )
      (norm2): RMSNorm()
      (mlp): MLP(
        (up): Linear(in_features=192, out_features=384, bias=False)
        (gate): Linear(in_features=192, out_features=384, bias=False)
        (down): Linear(in_features=384, out_features=192, bias=False)
        (act): SiLU()
      )
      (modulate): Linear(in_features=192, out_features=1152, bias=True)
    )
  )
  (norm): RMSNorm()
  (modulate): Linear(in_features=192, out_features=384, bias=True)
  (unpatch): UnPatch(
    (up): Linear(in_features=192, out_features=4, bias=True)

In [7]:
z = t.randn(100, 1, 28, 28, dtype=model.dtype, device=model.device)
y = t.ones(100, dtype=t.long, device=model.device)
for num in range(11):
  print(num)
  if num > 0:
    cfg = 3
  else:
    cfg = 0
  x_pred = sample(model, z, y*0 + num, cfg=cfg, n_steps=30)

  fig, axes = plt.subplots(4, 8, figsize=(8, 4))
  axes = axes.flatten()

  for i in range(32):
      img = x_pred[i].squeeze().detach().cpu().clamp(-1, 1)
      img = (img + 1) / 2  # map back to [0,1]

      axes[i].imshow(img, cmap='gray')
      axes[i].axis('off')

  plt.tight_layout()
  plt.show()



AttributeError: 'DiT' object has no attribute 'dtype'